In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import pickle

In [2]:
dataset=pd.read_csv("training_data.csv")
dataset.drop("Unnamed: 0",axis=1,inplace=True)
dataset.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,80.0,0,1,105.92,32.5,1,1,0,1,0,1,0,0,0,0,1,0
1,49.0,0,0,171.23,34.4,1,0,0,1,0,1,0,0,1,0,0,1
2,79.0,1,0,174.12,24.0,1,0,0,1,0,0,1,0,0,0,1,0
3,81.0,0,0,186.21,29.0,1,1,0,1,0,1,0,0,1,1,0,0
4,74.0,1,1,70.09,27.4,1,1,0,1,0,1,0,0,0,0,1,0


In [3]:
dataset.shape

(4187, 17)

#### Lets us fill BMI's null values with Median

In [4]:
dataset["bmi"].fillna(dataset["bmi"].median(),inplace=True)

In [5]:
dataset.isnull().sum()

age                               0
hypertension                      0
heart_disease                     0
avg_glucose_level                 0
bmi                               0
stroke                            0
gender_Male                       0
gender_Other                      0
ever_married_Yes                  0
work_type_Never_worked            0
work_type_Private                 0
work_type_Self-employed           0
work_type_children                0
Residence_type_Urban              0
smoking_status_formerly smoked    0
smoking_status_never smoked       0
smoking_status_smokes             0
dtype: int64

In [6]:
##Lets perform over sampling first be

In [7]:
smote=SMOTETomek(random_state=42,n_jobs=-1)

In [8]:
X=dataset.drop("stroke",axis=1)
y=dataset["stroke"]

In [9]:
X.columns

Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'gender_Male', 'gender_Other', 'ever_married_Yes',
       'work_type_Never_worked', 'work_type_Private',
       'work_type_Self-employed', 'work_type_children', 'Residence_type_Urban',
       'smoking_status_formerly smoked', 'smoking_status_never smoked',
       'smoking_status_smokes'],
      dtype='object')

In [10]:
x_,y_=smote.fit_resample(X,y)

In [11]:
x_.columns

Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'gender_Male', 'gender_Other', 'ever_married_Yes',
       'work_type_Never_worked', 'work_type_Private',
       'work_type_Self-employed', 'work_type_children', 'Residence_type_Urban',
       'smoking_status_formerly smoked', 'smoking_status_never smoked',
       'smoking_status_smokes'],
      dtype='object')

In [12]:
print(x_.shape,y_.shape)

(8006, 16) (8006,)


In [13]:
##Before oversampling
sampling={0:0,1:0}
for pt in y:
    if sampling[pt]==0:
        sampling[pt]=sampling[pt]+1
    else:
        sampling[pt]=sampling[pt]+1

print(sampling)

{0: 4035, 1: 152}


In [14]:
resampling={0:0,1:0} ##After oversampling
for pt in y_:
    if resampling[pt]==0:
        resampling[pt]=resampling[pt]+1
    else:
        resampling[pt]=resampling[pt]+1

print(resampling)

{0: 4003, 1: 4003}


In [15]:
scale=StandardScaler()##performing standard scaling
scaled_features=scale.fit_transform(x_)

In [16]:
x_.shape

(8006, 16)

In [17]:
scaled_features

array([[ 1.1248494 , -0.2994937 ,  5.44741494, ..., -0.36054534,
         1.56694932, -0.32150075],
       [-0.22486053, -0.2994937 , -0.18357331, ..., -0.36054534,
        -0.63818273,  3.11041262],
       [ 1.08131037,  3.33896843, -0.18357331, ..., -0.36054534,
         1.56694932, -0.32150075],
       ...,
       [ 1.09575621, -0.2994937 , -0.18357331, ..., -0.36054534,
         1.56694932, -0.32150075],
       [ 0.75952012, -0.2994937 , -0.18357331, ..., -0.36054534,
        -0.63818273, -0.32150075],
       [ 0.17444935, -0.2994937 , -0.18357331, ..., -0.36054534,
        -0.63818273, -0.32150075]])

In [18]:
colums=[]##Independent features
for feature in dataset.columns:
    if feature=="stroke":
        continue
    else:
        colums.append(feature)

In [19]:
scaled_data=pd.DataFrame(scaled_features,columns=colums)##Converting scaled data to dataframe

In [20]:
scaled_data.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,1.124849,-0.299494,5.447415,0.159705,0.914311,1.512576,-0.011177,0.669916,-0.050044,1.099341,-0.381893,-0.300977,-0.780957,-0.360545,1.566949,-0.321501
1,-0.224861,-0.299494,-0.183573,2.026632,1.282934,-0.661124,-0.011177,0.669916,-0.050044,1.099341,-0.381893,-0.300977,1.280481,-0.360545,-0.638183,3.110413
2,1.081310,3.338968,-0.183573,2.109244,-0.734795,-0.661124,-0.011177,0.669916,-0.050044,-0.909635,2.618534,-0.300977,-0.780957,-0.360545,1.566949,-0.321501
3,1.168388,-0.299494,-0.183573,2.454845,0.235267,1.512576,-0.011177,0.669916,-0.050044,1.099341,-0.381893,-0.300977,1.280481,2.773576,-0.638183,-0.321501
4,0.863615,3.338968,5.447415,-0.864518,-0.075152,1.512576,-0.011177,0.669916,-0.050044,1.099341,-0.381893,-0.300977,-0.780957,-0.360545,1.566949,-0.321501


In [21]:
scaled_data["stroke"]=y_
scaled_data.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,stroke
0,1.124849,-0.299494,5.447415,0.159705,0.914311,1.512576,-0.011177,0.669916,-0.050044,1.099341,-0.381893,-0.300977,-0.780957,-0.360545,1.566949,-0.321501,1
1,-0.224861,-0.299494,-0.183573,2.026632,1.282934,-0.661124,-0.011177,0.669916,-0.050044,1.099341,-0.381893,-0.300977,1.280481,-0.360545,-0.638183,3.110413,1
2,1.081310,3.338968,-0.183573,2.109244,-0.734795,-0.661124,-0.011177,0.669916,-0.050044,-0.909635,2.618534,-0.300977,-0.780957,-0.360545,1.566949,-0.321501,1
3,1.168388,-0.299494,-0.183573,2.454845,0.235267,1.512576,-0.011177,0.669916,-0.050044,1.099341,-0.381893,-0.300977,1.280481,2.773576,-0.638183,-0.321501,1
4,0.863615,3.338968,5.447415,-0.864518,-0.075152,1.512576,-0.011177,0.669916,-0.050044,1.099341,-0.381893,-0.300977,-0.780957,-0.360545,1.566949,-0.321501,1


In [22]:
#sns.pairplot(scaled_data,hue="stroke",palette="Set2")

## Lets perform train test split

In [23]:
# X=scaled_data.drop("stroke",axis=1)
# y=scaled_data["stroke"]
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)
X_train,X_test,y_train,y_test=train_test_split(x_,y_,test_size=0.3,random_state=42)

In [24]:
print(X_train.shape,X_test.shape)

(5604, 16) (2402, 16)


In [25]:
# accuracy=[]
# for i in range(1,80):
#     pipeline1=Pipeline([
#     ("scaling",StandardScaler()),
#     ("kmeans",KNeighborsClassifier(n_neighbors=i,n_jobs=-1))
# ])
#     pipeline1.fit(X_train,y_train)
#     y_pred=pipeline1.predict(X_test)
#     score=accuracy_score(y_test,y_pred)
#     accuracy.append(score)
    

## Plot K vs Accuracy

In [26]:
# plt.figure(figsize=(10,6))
# plt.plot(range(1,80),accuracy,color='blue', linestyle='dashed', 
#          marker='o',markerfacecolor='red', markersize=10)
# plt.title('Accuracy Rate vs. K Value')
# plt.xlabel('K')
# plt.ylabel('Accuracy Rate')

## Error Rate vs K

In [27]:
# error_rate = []
# for i in range(1,80):
#  knn = KNeighborsClassifier(n_neighbors=i,n_jobs=-1)
#  knn.fit(X_train,y_train)
#  pred_i = knn.predict(X_test)
#  error_rate.append(np.mean(pred_i != y_test))

# plt.figure(figsize=(10,6))
# plt.plot(range(1,80),error_rate,color='blue', linestyle='dashed', 
#          marker='o',markerfacecolor='red', markersize=10)
# plt.title('Error Rate vs. K Value')
# plt.xlabel('K')
# plt.ylabel('Error Rate')
# print("Minimum error:-",min(error_rate),"at K =",error_rate.index(min(error_rate)))

In [28]:
## As we can see that after k=65 there is not much change in error rate and accuracy
##if we choose lower value model might overfit

In [29]:
# knn=KNeighborsClassifier(n_neighbors=55,n_jobs=-1)

In [30]:
pipeline=Pipeline([
    ("scaling",StandardScaler()),
    ("kmeans",KNeighborsClassifier(n_neighbors=55,n_jobs=-1))
])

In [31]:
y_train.value_counts()

1    2810
0    2794
Name: stroke, dtype: int64

In [32]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('scaling', StandardScaler()),
                ('kmeans', KNeighborsClassifier(n_jobs=-1, n_neighbors=55))])

In [33]:
score=cross_val_score(pipeline,X_train,y_train,cv=10,n_jobs=-1)

In [34]:
##Lets See how our model performs on test data

In [35]:
score

array([0.85204991, 0.85383244, 0.86809269, 0.85918004, 0.86607143,
       0.9       , 0.84642857, 0.84642857, 0.86964286, 0.86428571])

In [36]:
score.mean()

0.8626012223071046

In [37]:
 print("The max accuracy we can get is {} and the lowest it can go is {}".format(score.max(),score.min()))

The max accuracy we can get is 0.9 and the lowest it can go is 0.8464285714285714


In [38]:
y_test.value_counts()

0    1209
1    1193
Name: stroke, dtype: int64

In [39]:
pipeline.fit(X_train,y_train)
y_predicted=pipeline.predict(X_test)
cm=confusion_matrix(y_test,y_predicted)

In [40]:
cm

array([[ 956,  253],
       [ 111, 1082]])

In [41]:
pred=pipeline.predict(x_[:500])

In [42]:
cmm=confusion_matrix(y_[:500],pred)
print(cmm)

[[279  70]
 [ 78  73]]


In [43]:
print(classification_report(y_test,y_predicted))

              precision    recall  f1-score   support

           0       0.90      0.79      0.84      1209
           1       0.81      0.91      0.86      1193

    accuracy                           0.85      2402
   macro avg       0.85      0.85      0.85      2402
weighted avg       0.85      0.85      0.85      2402



In [44]:
# def predictProbab(params):
#     scaledlst=[]
#     for feature in params:
#         scaledlst.append(feature)
#     scaledValues=np.array(scaledlst)
#     vals=scale.fit_transform([scaledValues])
#     output=knn.predict_proba(vals)
#     return output
        
        

In [45]:
# def predictProbab(age,hypertension,heart_disease,avg_glucose_lvl,bmi,isMale,isOther,isMarried,neverWorked,privateWork,selfEmp,childerWork,isUrban,formerlySmokes,neverSmoke,smoke):
    

In [46]:
# para=[61,1,1,241,26,1,0,1,1,0,0,0,0,1,1,1]

In [47]:
# print(predictProbab(para))

In [48]:
# def predictOP(params):
#     scaledlst=[]
#     for feature in params:
#         scaledlst.append(feature)
#     scaledValues=np.array(scaledlst)
#     print(scaledValues)
#     vals=scale.fit_transform([scaledValues])
#     print(vals)
#     output=knn.predict(vals)
#     return output
        
        

In [49]:
# print(predictOP(para))

In [50]:
# model=open("model.pkl","wb")
# pickle.dump(pipeline,model)
# model.close()

In [51]:
model=pickle.load(open("model.pkl","rb"))

In [67]:
pred=model.predict(x_[:1])

In [68]:
confusion_matrix(y_[:1],pred)

array([[1]])

In [70]:
print(classification_report(y_[:1],pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

